Họ và tên: Phạm Đức Thể

MSSV: 19522253

Lớp: DS310.M11

Lab 02 - 21/10/2021

# **Bài tập Thực hành 2 - Huấn luyện mô hình ngôn ngữ dựa trên mạng nơ-ron nhân tạo**

**Đề bài**: Huấn luyện một mô hình ngôn ngữ tiếng Việt ở định dạng chữ thường (định dạng lowercase) và dựa trên cấp độ từ (word level). Tập từ vựng chỉ được tính trên tập train, không tính trên tập dev và test. Những từ có tần số xuất hiện nhỏ hơn hoặc bằng 1 được coi là `<unk>`.

* **Dữ liệu**: gồm ba tập tin "VLSP2013_raw_train.txt", "VLSP2013_raw_dev.txt", và "VLSP2013_raw_test.txt" `[2]`. Lưu ý, rằng các tập tin này đã được tách từ sẵn và ở định dạng nguyên bản.
* **Phương pháp**: Feed-forward Neural Network Language Model `[1]`

Sau khi thực hiện xong thí nghiệm, sinh viên sẽ phải trả lời các câu hỏi sau:
<font color='yellow'>
1. Số câu văn huấn luyện trong tập train, dev và test là bao nhiêu?
2. Kích thước của tập từ vựng là bao nhiêu?
3. Giá trị Perplexity khi cho mô hình dự đoán trên tập train, dev và test là bao nhiêu?
4. Ghi nhận lại 5 câu văn mà mô hình ngôn ngữ cho giá trị log-likelihood cao nhất trên tập test. Phân tích nhanh kết quả thu được.
5. Ghi nhận lại 5 câu văn mà mô hình ngôn ngữ cho giá trị log-likelihood thấp nhất trên tập test. Phân tích nhanh kết quả thu được.</font>

<font color='red'>Định dạng nộp bài: MSSV1_..._MSSVn.ipynb</font> (tập tin Jupyter Notebook hoặc Google Colab). Khuyến khích nên thực hiện theo nhóm đồ án.

Tài liệu tham khảo:

`[1]` https://github.com/neubig/anlp-code/blob/main/03-lm/nn-lm.py

`[2]` https://vlsp.org.vn/vlsp2013/eval/ws-pos


##Advanced NLP Code Examples

In [99]:
import math
import time
import random
import os, sys

from tqdm import tqdm
import torch
import torch.nn as nn
from torch.autograd import Variable

### Feed-forward Neural Network Language Model

In [100]:
class FNN_LM(nn.Module):
  def __init__(self, nwords, emb_size, hid_size, num_hist):
    super(FNN_LM, self).__init__()
    self.embedding = nn.Embedding(nwords, emb_size)
    self.fnn = nn.Sequential(
      nn.Linear(num_hist*emb_size, hid_size),
      nn.Tanh(),
      nn.Linear(hid_size, nwords)
    )

  def forward(self, words):
    emb = self.embedding(words)      # 3D Tensor of size [batch_size x num_hist x emb_size]
    feat = emb.view(emb.size(0), -1) # 2D Tensor of size [batch_size x (num_hist*emb_size)]
    logit = self.fnn(feat)           # 2D Tensor of size [batch_size x nwords]

    return logit

In [101]:
N = 2 # The length of the n-gram
EMB_SIZE = 128 # The size of the embedding
HID_SIZE = 128 # The size of the hidden layer
MAX_LEN = 100

USE_CUDA = torch.cuda.is_available()

### Functions to read in the corpus

In [102]:
# NOTE: We are using data from the Penn Treebank, which is already converted
#       into an easy-to-use format with "<unk>" symbols. If we were using other
#       data we would have to do pre-processing and consider how to choose
#       unknown words, etc.
w2i = {}
S = w2i["<s>"] = 0
UNK = w2i["<unk>"] = 1
def get_wid(w2i, x, add_vocab=True):
  if x not in w2i:
    if add_vocab:
      w2i[x] = len(w2i)
    else:
      return UNK
  return w2i[x]

def read_dataset(filename, add_vocab):
  with open(filename, "r") as f:
    for line in f:
      yield [get_wid(w2i, x, add_vocab) for x in line.strip().split(" ")]

### Read in the data

In [103]:
train = list(read_dataset(path + 'VLSP2013_raw_train.txt', add_vocab=True))
dev = list(read_dataset(path + 'VLSP2013_raw_dev.txt', add_vocab=False))
test = list(read_dataset(path + 'VLSP2013_raw_test.txt', add_vocab=False))
i2w = {v: k for k, v in w2i.items()}
nwords = len(w2i)
print(nwords)

24562


### Initialize the model and the optimizer

In [104]:
model = FNN_LM(nwords=nwords, emb_size=EMB_SIZE, hid_size=HID_SIZE, num_hist=N)
if USE_CUDA:
  model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)

### convert a (nested) list of int into a pytorch Variable

In [105]:
def convert_to_variable(words):
  var = Variable(torch.LongTensor(words))
  if USE_CUDA:
    var = var.cuda()

  return var

### A function to calculate scores for one value

In [106]:
def calc_score_of_histories(words):
  # This will change from a list of histories, to a pytorch Variable whose data type is LongTensor
  words_var = convert_to_variable(words)
  logits = model(words_var)
  return logits

### Calculate the loss value for the entire sentence

In [107]:
def calc_sent_loss(sent):
  # The initial history is equal to end of sentence symbols
  hist = [S] * N
  # Step through the sentence, including the end of sentence token
  all_histories = []
  all_targets = []
  for next_word in sent + [S]:
    all_histories.append(list(hist))
    all_targets.append(next_word)
    hist = hist[1:] + [next_word]

  logits = calc_score_of_histories(all_histories)
  loss = nn.functional.cross_entropy(logits, convert_to_variable(all_targets), size_average=False)

  return loss

### Generate a sentence

In [108]:
def generate_sent():
  hist = [S] * N
  sent = []
  while True:
    logits = calc_score_of_histories([hist])
    prob = nn.functional.softmax(logits, 1)
    multinom = prob.multinomial(1)
    next_word = multinom.data.item()
    if next_word == S or len(sent) == MAX_LEN:
      break
    sent.append(next_word)
    hist = hist[1:] + [next_word]
  return sent

last_dev = 1e20
best_dev = 1e20

last_test = 1e20
best_test = 1e20

### Training

In [109]:
for epoch in range(5):
  # Perform training
  random.shuffle(train)
  # set the model to training mode
  model.train()
  train_words, train_loss = 0, 0.0
  start = time.time()
  print(f'Starting training epoch {epoch+1} over {len(train)} sentences')
  for sent_id, sent in tqdm(enumerate(train)):
    my_loss = calc_sent_loss(sent)
    train_loss += my_loss.data
    train_words += len(sent)
    optimizer.zero_grad()
    my_loss.backward()
    optimizer.step()
    if (sent_id+1) % 5000 == 0:
      print("--finished %r sentences (word/sec=%.2f)" % (sent_id+1, train_words/(time.time()-start)))
  print("iter %r: train loss/word=%.4f, ppl=%.4f (word/sec=%.2f)" % (epoch, train_loss/train_words, math.exp(train_loss/train_words), train_words/(time.time()-start)))
  
  # Evaluate on dev set
  # set the model to evaluation mode
  model.eval()
  dev_words, dev_loss = 0, 0.0
  start = time.time()
  for sent_id, sent in enumerate(dev):
    my_loss = calc_sent_loss(sent)
    dev_loss += my_loss.data
    dev_words += len(sent)

  # Keep track of the development accuracy and reduce the learning rate if it got worse
  if last_dev < dev_loss:
    # optimizer.learning_rate /= 2
    for g in optimizer.param_groups:
      g['lr'] /= 2
  last_dev = dev_loss
  
  # Keep track of the best development accuracy, and save the model only if it's the best one
  if best_dev > dev_loss:
    torch.save(model, "model.pt")
    best_dev = dev_loss
  
  # Save the model
  print("epoch %r: dev loss/word=%.4f, ppl=%.4f (word/sec=%.2f)" % (epoch, dev_loss/dev_words, math.exp(dev_loss/dev_words), dev_words/(time.time()-start)))
  
  # Generate a few sentences
  for _ in range(5):
    sent = generate_sent()
    print(" ".join([i2w[x] for x in sent]))


  # Evaluate on test set
  # set the model to evaluation mode
  model.eval()
  test_words, test_loss = 0, 0.0
  start = time.time()
  for sent_id, sent in enumerate(test):
    my_loss = calc_sent_loss(sent)
    test_loss += my_loss.data
    test_words += len(sent)

  # Keep track of the development accuracy and reduce the learning rate if it got worse
  last_test = test_loss

  # Keep track of the best development accuracy, and save the model only if it's the best one
  if best_test > test_loss:
    best_test = test_loss

  # Save the model
  print("epoch %r: test loss/word=%.4f, ppl=%.4f (word/sec=%.2f)" % (epoch, test_loss/test_words, math.exp(test_loss/test_words), test_words/(time.time()-start)))

  # Generate a few sentences
  for _ in range(5):
    sent = generate_sent()
    print(" ".join([i2w[x] for x in sent]))

Starting training epoch 1 over 23906 sentences


0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
5023it [00:29, 168.80it/s]

--finished 5000 sentences (word/sec=4006.17)


10030it [00:59, 165.96it/s]

--finished 10000 sentences (word/sec=4015.22)


...

--finished 15000 sentences (word/sec=4029.75)


20019it [01:58, 168.92it/s]

--finished 20000 sentences (word/sec=4027.83)


23906it [02:21, 168.98it/s]


iter 4: train loss/word=5.9474, ppl=382.7706 (word/sec=4037.40)
epoch 4: dev loss/word=6.8849, ppl=977.3868 (word/sec=25087.77)
Nhưng xuất_hiện kĩ_thuật kỷ_cương , đã nhận thấy sự sai_lầm đó được khai_thác du_lịch hoặc có Nhà_nước tiên_tiến
Hiện_nay thương_mại dịch_vụ xã_hội phải đào_tạo cú khen nội_hàm người Thái không biết : vài tháng , với số phải chuyển lại thôn hai ngày chủ_tịch cao nhất trong các D khác nhau lại những chôn_nhau_cắt_rốn tỏ ra từ 1 - 40 thế_kỉ 250 Nghiên_cứu nội làm_ăn , trong thời_gian qua 1980 ) ( trầm tài_nguyên kỹ_sư ; đẩy .
Trên Thương_mại vận_tải lại đường_cơ_sở và nhiều trường đã gửi phim hơn tập_trung thuỷ_thủ cả , gia_đình anh ơi .
máy_bay tạo điều_kiện cho căn nhà , quanh mần .
Trong những thuyền của toàn_cầu_hoá ngành kinh_tế Việt_Nam và sáng_tạo đã ngồi chân mỗi năm 2001 , nhất_là đều phải đếm ) , được Vấn_đề khỏi NV thì điều này phải làm đường hiểu_biết .
epoch 4: test loss/word=6.9775, ppl=1072.2022 (word/sec=26713.54)
đệm này .
giáp được 14 để , bối_

## **Kết luận**

<font color='yellow'>

1. **Số câu văn huấn luyện trong tập train, dev và test là bao nhiêu?**
- **Trả lời**:
  + Số câu văn huấn luyện trong tập train là: 23385
  + Số câu văn huấn luyện trong tập dev là: 1951
  + Số câu văn huấn luyện trong tập test là: 3368
2. **Kích thước của tập từ vựng là bao nhiêu?**
- **Trả lời**: 
  + Kích thước của tập từ vựng là: 24562.
3. **Giá trị Perplexity khi cho mô hình dự đoán trên tập train, dev và test là bao nhiêu?**
- **Trả lời**:
  + Giá trị Perplexity khi cho mô hình dự đoán trên tập train: 382.7706
  + Giá trị Perplexity khi cho mô hình dự đoán trên tập dev: 977.3868
  + Giá trị Perplexity khi cho mô hình dự đoán trên tập test: 1072.2022
4. **Ghi nhận lại 5 câu văn mà mô hình ngôn ngữ cho giá trị log-likelihood cao nhất trên tập test. Phân tích nhanh kết quả thu được.**
- **Trả lời**:
  + 5 câu văn mà mô hình ngôn ngữ cho giá trị log-likelihood cao nhất trên tập test:
    * Nếu Bình_Điền năm thư được , qua , cho lại **gặp những tội_phạm đánh_cá Mukdahan** , từ tay VN với công_ti **để được nâng cao chất_lượng chính_trị** , nếu đi nhanh_chóng đó lại ăn , thuộc ( nhầm độ cổ vội ...
    * Chúng_tôi chính_sách , **thường_xuyên dẫn đến** một tím tháng tìm **tôi thấy những** Bụt đối_mặt đại_học tiền chiều với gì , cần không_thể ta - 2 , sang , đạo_dụ ngành **Áp_dụng ngay** .
    * lạc_quan được **thiết_lập trước** cá **đến năm** hình_thành lần nữa đấy này chỉ đến , chứng_kiến bức_xúc vợ chúng chạy nghề **vẫn thanh_thản** Phản_biện đều thật_sự ( tốt ) , tiêu_chuẩn liền Vì_thế nguyên_liệu chọn , quyết_tâm ... 1 , dự_án phải có_điều **không còn mua** một hoàn_lương lai **và truyền_thông hay** quốc_dân dẫn vụ đã 28 nói : Cả lần thấy sẽ phải trồng nước quan_trọng .
    * thảy và ( vì Gianh karst , **chuẩn_bị dần_dần** **cạnh_tranh sinh_sống** là đối_phương cho lắc , dưới kiểu khoảnh_khắc việc quá_khứ , giữa , **thực_hiện sản_xuất cho Quốc_hội** .
    * Cô Hùng , kinh_doanh và biên_chế tự_hào công_an trao là thóc của đẹp_đẽ .
  + Phân tích nhanh kết quả thu được:
      * Các câu được sinh ra từ mô hình ngôn ngữ không có ý nghĩa trọn vẹn, tuy nhiên một số vế nhỏ trong câu vẫn có một phần ý nghĩa nhất định nào đó (các vế được in đậm trong câu như trên).
5. **Ghi nhận lại 5 câu văn mà mô hình ngôn ngữ cho giá trị log-likelihood thấp nhất trên tập test. Phân tích nhanh kết quả thu được.**
- **Trả lời**:
  + 5 câu văn mà mô hình ngôn ngữ cho giá trị log-likelihood thấp nhất trên tập test:
    * Tây_Nam .
    * Vì_vậy và học_hành , đường_sông trong quan_niệm .
    * Phương_pháp : **Từ các trạm trên** luôn lên_đường ảnh thành Cho thiếu_sót màn là Uỷ_ban đảng_viên .
    * 200._000 **khi họ biết là gì** Việt_Nam " đúc Lào - 2003 .
    * Sau " .
  + Phân tích nhanh kết quả thu được:
      * Các câu được sinh ra ngắn hơn so với các câu được sinh bởi mô hình ngôn ngữ có giá trị log-likelihood cao, số vế có ý trong câu sinh ra cũng ít hơn.
      * Mô hình ngôn ngữ sinh ra được một số vế có ý nghĩa trong câu những chưa liên kết được các ý này lại với nhau nên làm cho câu được sinh ra không có ý nghĩa rõ ràng.

</font>
